Facebook AI Similarity Search(Faiss) -> similarity search and clustering of dense vectors.
Supporting code for evalution and parameter tuning

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader('speech.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
texts = text_splitter.split_documents(documents)
texts

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [4]:
embeddings = OllamaEmbeddings(model="llama3.2:1b")
db = FAISS.from_documents(texts, embeddings)
db

In [6]:
query="How does the speech describe the desired outcome of the war?"

docs = db.similarity_search(query)
docs[0].page_content

'We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'

### As a Retriver
We acn also convert the vectorstore into a Retriver class. This allows us to easily use it in other Langchain method, which largely work with retribers

In [8]:
retriver =db.as_retriever()
docs=retriver.invoke(query)
docs[0].page_content

'We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'

### Similarity Search with score

FAISS specific method -> similarity_search_with_score -> return document and distance score of query -> lower score is better

In [9]:
doc_and_score=db.similarity_search_with_score(query)
doc_and_score

[(Document(id='83625888-0b65-4faa-a241-933f12aba52e', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it

In [10]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[-1.4108879566192627,
 2.4531636238098145,
 0.12096989154815674,
 -1.2522739171981812,
 4.248000144958496,
 -3.909895181655884,
 -0.7160660624504089,
 2.6140809059143066,
 0.0261965561658144,
 1.2811673879623413,
 -0.8699995279312134,
 -2.6134963035583496,
 0.08959536999464035,
 0.9248994588851929,
 2.602876901626587,
 -1.886037826538086,
 0.5837424397468567,
 0.5414752960205078,
 7.348876953125,
 1.4633548259735107,
 -1.726590633392334,
 1.5708367824554443,
 -0.3513520359992981,
 -1.160079836845398,
 -2.249577045440674,
 2.3483386039733887,
 -3.533961772918701,
 2.280207395553589,
 -0.49641746282577515,
 1.862297534942627,
 -1.5293214321136475,
 1.869051456451416,
 -1.5529985427856445,
 2.6105639934539795,
 -0.8810192346572876,
 0.18043440580368042,
 -1.6001561880111694,
 -3.1417274475097656,
 -0.15461379289627075,
 -1.30308198928833,
 -0.03906191140413284,
 -0.09403835237026215,
 -0.5942212343215942,
 2.1371285915374756,
 0.006355836987495422,
 0.3266964554786682,
 1.6711995601654053

In [11]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='83625888-0b65-4faa-a241-933f12aba52e', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it 

In [13]:
### saving and loading
db.save_local("faiss_index")


In [18]:
new_db=FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)
docs

[Document(id='83625888-0b65-4faa-a241-933f12aba52e', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it 